In [1]:
# Some standard imports
import io
import numpy as np
import torch
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

/home/mazheng/miniconda3/envs/eff/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys 
sys.path.append("..")
from utils_inference import BBoxTransform,ClipBoxes,postprocess

In [3]:
sys.path.append("../..")
from backbone import EfficientDetBackbone
from utils.vis_utils import model_params, model_load
from utils.eff_utils import load_yaml
from dataclasses import dataclass
from utils.utils import preprocess

In [4]:
from pathlib import Path
path = Path.cwd().parents[1]

In [5]:
project_name = "0509split"
# project_name = "4-4"
# project_name = "detection-untunnel"
compound_coef = 2
threshold = 0.3
iou_threshold = 0.2
# model_path = 'logs/0509split/efficientdet-d2_71_145500.pth'
model_path = str(path/'logs/0509split_d2/efficientdet-d2_125_253500.pth')

In [6]:
ann_json = path / 'datasets/{}/annotations/instances_val.json'.format(project_name)
# ann_json = path / 'datasets/val_dataset/0409/0409notunnel/instance_val.json'
img_path = path / 'datasets/{}/val'.format(project_name)
# img_path = Path('datasets/val_dataset/0409/0409notunnel/img')
# img_path = Path('datasets/4-4')
yaml_path = path / 'projects/{}.yml'.format(project_name)
# yaml_path = path / 'projects/{}_3anchor.yml'.format(project_name)
# yaml_path = path / 'projects/0414split.yml'

project_params = load_yaml(str(yaml_path))

ratios=eval(project_params['anchors_ratios'])
scales=eval(project_params['anchors_scales'])
# scales=[(2 ** 0)/2, (2 ** (1.0 / 3.0))/2, (2 ** (2.0 / 3.0))/2]
# ratios=[(0.6, 1.6), (1.2, 0.8), (1.7, 0.6)]
obj_list = project_params['obj_list']
params = model_params(compound_coef, obj_list, ratios, scales, model_path)

In [7]:
params

model_params(compound_coef=2, obj_list=['pounding', 'pothole', 'hcrack', 'rsign', 'vcrack', 'animal', 'csign', 'people', 'indicator-red', 'lcrack', 'spiledmaterial', 'tsign', 'gantry', 'osign', 'bsign', 'label', 'fracturing', 'indicator-green', 'indicator-off', 'light-off'], ratios=[(0.7, 1.5), (0.9, 1.1), (1.2, 0.8), (1.7, 0.6), (3.3, 0.3)], scales=[0.35714285714285715, 0.44997180353388333, 0.5669289471314998], model_path='/home/mazheng/waste-detection/Yet-Another-EfficientDet-Pytorch/logs/0509split_d2/efficientdet-d2_125_253500.pth')

In [8]:
import torch.nn as nn

In [9]:
from torchvision.ops.boxes import batched_nms

In [11]:
torch.Tensor().numpy()

array([], dtype=float32)

In [12]:
def postprocess(
    x,
    anchors,
    regression,
    classification,
    regressBoxes,
    clipBoxes,
    threshold,
    iou_threshold,
):
    transformed_anchors = regressBoxes(anchors, regression)
    transformed_anchors = clipBoxes(transformed_anchors, x)
    scores = torch.max(classification, dim=2, keepdim=True)[0]
    # print(scores)
    # print(scores > threshold)
    scores_over_thresh = (scores > threshold)[:, :, 0]
    class_ids_list = []
    rois_list = []
    scores_list = []
    for i in range(x.shape[0]):
        if scores_over_thresh[i].sum() == 0:
            # print(scores_over_thresh[i].sum())
            # rois_list.append(torch.Tensor([])),
            # class_ids_list.append(torch.Tensor([])),
            # scores_list.append(torch.Tensor([]))
            continue

        classification_per = classification[i, scores_over_thresh[i, :], ...].permute(
            1, 0
        )
        transformed_anchors_per = transformed_anchors[i, scores_over_thresh[i, :], ...]
        scores_per = scores[i, scores_over_thresh[i, :], ...]
        scores_, classes_ = classification_per.max(dim=0)
        anchors_nms_idx = batched_nms(
            transformed_anchors_per,
            scores_per[:, 0],
            classes_,
            iou_threshold=iou_threshold,
        )

        if anchors_nms_idx.shape[0] != 0:
            classes_ = classes_[anchors_nms_idx]
            scores_ = scores_[anchors_nms_idx]
            boxes_ = transformed_anchors_per[anchors_nms_idx, :]

            rois_list.append(boxes_.cpu()),
            class_ids_list.append(scores_),
            scores_list.append(boxes_)
        # else:
            # rois_list.append(torch.Tensor([])),
            # class_ids_list.append(torch.Tensor([])),
            # scores_list.append(torch.Tensor([]))

    return rois_list, class_ids_list, scores_list

In [17]:
class whole(nn.Module):
    def __init__(
        self,
        num_classes=80,
        compound_coef=0,
        load_weights=False,
        onnx_export=False,
        **kwargs
    ):
        super(whole, self).__init__()

        self.effdet = EfficientDetBackbone(
                    onnx_export=onnx_export,
                    compound_coef=compound_coef,
                    num_classes=num_classes,
                    # replace this part with your project's anchor config
                    ratios=kwargs.get("ratios", [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]),
                    scales=kwargs.get("scales", [2**0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]),
                )
        self.regressBoxes = BBoxTransform()
        self.clipBoxes = ClipBoxes()

    def forward(self,x):
        _, regression, classification, anchors=self.effdet(x)

        out = postprocess(
            x,
            anchors,
            regression,
            classification,
            self.regressBoxes,
            self.clipBoxes,
            threshold,
            iou_threshold,
        )
        
        return out



In [18]:
Whole = whole(onnx_export=True,
                compound_coef=params.compound_coef,
                num_classes=len(params.obj_list),
                # replace this part with your project's anchor config
                ratios=params.ratios,
                scales=params.scales,)

In [19]:
Whole.effdet.load_state_dict(torch.load(params.model_path))

<All keys matched successfully>

In [20]:
Whole.eval()

whole(
  (effdet): EfficientDetBackbone(
    (bifpn): Sequential(
      (0): BiFPN(
        (conv6_up): SeparableConvBlock(
          (depthwise_conv): Conv2dStaticSamePadding(
            (conv): Conv2d(112, 112, kernel_size=(3, 3), stride=(1, 1), groups=112, bias=False)
          )
          (pointwise_conv): Conv2dStaticSamePadding(
            (conv): Conv2d(112, 112, kernel_size=(1, 1), stride=(1, 1))
          )
          (bn): BatchNorm2d(112, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
        (conv5_up): SeparableConvBlock(
          (depthwise_conv): Conv2dStaticSamePadding(
            (conv): Conv2d(112, 112, kernel_size=(3, 3), stride=(1, 1), groups=112, bias=False)
          )
          (pointwise_conv): Conv2dStaticSamePadding(
            (conv): Conv2d(112, 112, kernel_size=(1, 1), stride=(1, 1))
          )
          (bn): BatchNorm2d(112, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
        (conv4_up): Sepa

In [90]:
batch_size = 1
x = torch.randn(batch_size, 3, 768, 768, requires_grad=True)
torch.onnx.export(Whole, x,
                  'effdet.onnx',
                  verbose=False,
                  input_names=['data'],
                  opset_version=11)

../../efficientnet/utils_extra.py:36: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  extra_h = (math.ceil(w / self.stride[1]) - 1) * self.stride[1] - w + self.kernel_size[1]
../../efficientnet/utils_extra.py:37: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  extra_v = (math.ceil(h / self.stride[0]) - 1) * self.stride[0] - h + self.kernel_size[0]
../../efficientnet/utils_extra.py:75: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a con

In [86]:
img_name = "DLLJ1796.jpg"
str(img_path/img_name)

'/home/mazheng/waste-detection/Yet-Another-EfficientDet-Pytorch/datasets/0509split/val/DLLJ1796.jpg'

In [87]:
input_sizes=[512, 640, 768, 896, 1024, 1280, 1280, 1536]
input_size = (
        input_sizes[2]
    )

ori_imgs, framed_imgs, framed_metas = preprocess(
    str(img_path/img_name),
    max_size=input_size,
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225],
)
x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)
x = x.permute(0, 3, 1, 2)

In [21]:
batch_size = 1
x = torch.randn(batch_size, 3, 768, 768, requires_grad=True)
# features, regression, classification, anchors = model(x)
# print([i.shape for i in torch_out])
# Export the model


In [22]:
output = Whole(x)

In [24]:
output

([], [], [])

In [23]:
len(output)

3

In [91]:
import onnx

# Load the ONNX model
model_ = onnx.load("effdet.onnx")

# Check that the model is well formed
onnx.checker.check_model(model_)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model_.graph))

graph torch-jit-export (
  %data[FLOAT, 1x3x768x768]
) initializers (
  %effdet.backbone_net.model._blocks.0._bn1.bias[FLOAT, 32]
  %effdet.backbone_net.model._blocks.0._bn1.num_batches_tracked[INT64, scalar]
  %effdet.backbone_net.model._blocks.0._bn1.running_mean[FLOAT, 32]
  %effdet.backbone_net.model._blocks.0._bn1.running_var[FLOAT, 32]
  %effdet.backbone_net.model._blocks.0._bn1.weight[FLOAT, 32]
  %effdet.backbone_net.model._blocks.0._bn2.bias[FLOAT, 16]
  %effdet.backbone_net.model._blocks.0._bn2.num_batches_tracked[INT64, scalar]
  %effdet.backbone_net.model._blocks.0._bn2.running_mean[FLOAT, 16]
  %effdet.backbone_net.model._blocks.0._bn2.running_var[FLOAT, 16]
  %effdet.backbone_net.model._blocks.0._bn2.weight[FLOAT, 16]
  %effdet.backbone_net.model._blocks.0._depthwise_conv.conv.weight[FLOAT, 32x1x3x3]
  %effdet.backbone_net.model._blocks.0._project_conv.conv.weight[FLOAT, 16x32x1x1]
  %effdet.backbone_net.model._blocks.0._se_expand.conv.bias[FLOAT, 32]
  %effdet.backbone_n

In [92]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("effdet.onnx", None, ['CUDAExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

In [93]:
output[2].numpy()

AttributeError: 'list' object has no attribute 'numpy'

In [94]:
# features, regression, classification, anchors
np.testing.assert_allclose(to_numpy(output[2]), ort_outs[2], rtol=1e-03, atol=1e-05)

AttributeError: 'list' object has no attribute 'requires_grad'